In [1]:
!pip install qiskit qiskit-aer

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.0/8.0 MB 33.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 84.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 88.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.0 MB/s eta 0:00:00


In [15]:
# Quantum Phase Estimation (QPE) using Qiskit 2.x

from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator
from qiskit.visualization import plot_histogram
import numpy as np
import matplotlib.pyplot as plt

def qpe_circuit(num_count_qubits, theta):
    """Constructs the Quantum Phase Estimation (QPE) circuit."""
    qc = QuantumCircuit(num_count_qubits + 1, num_count_qubits)

    # Step 1: Apply Hadamard gates on counting qubits
    qc.h(range(num_count_qubits))

    # Step 2: Prepare eigenstate |1> for the target qubit
    qc.x(num_count_qubits)

    # Step 3: Apply controlled-U operations (phase rotations)
    for qubit in range(num_count_qubits):
        qc.cp(2 * np.pi * theta * (2 ** qubit), qubit, num_count_qubits)

    # Step 4: Apply inverse QFT
    inverse_qft(qc, num_count_qubits)

    # Step 5: Measure counting qubits
    qc.measure(range(num_count_qubits), range(num_count_qubits))

    return qc

In [16]:
def inverse_qft(qc, n):
    """Apply the inverse Quantum Fourier Transform on n qubits."""
    for qubit in range(n // 2):
        qc.swap(qubit, n - qubit - 1)
    for j in range(n):
        for k in range(j):
            qc.cp(-np.pi / 2 ** (j - k), k, j)
        qc.h(j)
    return qc

In [19]:
def run_qpe(num_count_qubits=4, theta=0.375):
    """Executes QPE, visualizes the circuit (text-based), and compares theoretical output."""
    simulator = AerSimulator()

    # Build QPE circuit
    qc = qpe_circuit(num_count_qubits, theta)

    # Visualize circuit structure (text only, avoids pylatexenc dependency)
    print(f"\n🧠 QPE Circuit for θ = {theta} with {num_count_qubits} counting qubits:")
    print(qc.draw('text'))  # use text mode instead of 'mpl'

    # Run simulation (no noise)
    compiled_circuit = transpile(qc, simulator)
    result = simulator.run(compiled_circuit, shots=2048).result()
    counts = result.get_counts()

    # Plot results
    plot_histogram(counts, title=f"QPE Output (θ={theta}, qubits={num_count_qubits})")
    plt.show()

    # Theoretical phase comparison
    theoretical_binary = format(int(theta * (2 ** num_count_qubits)), f'0{num_count_qubits}b')
    theoretical_decimal = int(theoretical_binary, 2) / (2 ** num_count_qubits)

    print(f"🎯 Theoretical binary phase: {theoretical_binary}")
    print(f"🎯 Theoretical decimal phase: {theoretical_decimal:.4f}")

    print("\n📊 Measured results:")
    for k, v in sorted(counts.items(), key=lambda x: x[1], reverse=True):
        print(f"  {k}: {v} counts")

    return counts


In [20]:
if __name__ == "__main__":
    # Define test parameters
    phase_values = [0.25, 0.375, 0.5]   # Different theta values
    qubit_counts = [4, 5]               # 4 or 5 counting qubits for higher precision

    # Run QPE for all combinations
    for n_count in qubit_counts:
        for theta in phase_values:
            print(f"\n🚀 Running QPE with {n_count} counting qubits and θ = {theta}")
            run_qpe(num_count_qubits=n_count, theta=theta)


🚀 Running QPE with 4 counting qubits and θ = 0.25

🧠 QPE Circuit for θ = 0.25 with 4 counting qubits:
     ┌───┐                                   ┌───┐                         »
q_0: ┤ H ├─■───────────────────────────────X─┤ H ├─■──────────────■────────»
     ├───┤ │                               │ └───┘ │P(-π/2) ┌───┐ │        »
q_1: ┤ H ├─┼────────■────────────────X─────┼───────■────────┤ H ├─┼────────»
     ├───┤ │        │                │     │                └───┘ │P(-π/4) »
q_2: ┤ H ├─┼────────┼──────■─────────X─────┼──────────────────────■────────»
     ├───┤ │        │      │               │                               »
q_3: ┤ H ├─┼────────┼──────┼───────■───────X───────────────────────────────»
     ├───┤ │P(π/2)  │P(π)  │P(2π)  │P(4π)                                  »
q_4: ┤ X ├─■────────■──────■───────■───────────────────────────────────────»
     └───┘                                                                 »
c: 4/═════════════════════════════════════════════